# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install gradio==6.1.0 openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

## 1.3 LangSmith 环境配置
我们需要先前往 LangSmith 的官网并进行注册登录。

登录后我们就进入了下面这个初始界面，此时我们需要找到左下角的 Setting ，然后在里面先获取新建一个 API Key。

创建完成后，我们就可以将其配置到环境变量中。除了 API_Key 以外，通常 LangSmith 的项目还需要设置是否跟踪、上传地址以及项目名称信息（这个需要自定义设置）。

In [ ]:
import os
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "your_langsmith_api_key"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "ai-studio-evaluation"

# 2. 大模型评估

## 2.1 大模型评估简介

在构建和迭代大模型应用时，我们往往会频繁修改提示词（Prompt）、替换底层模型、或调整系统架构。但一个常被忽视的问题是：这些改动究竟让应用变得更好，还是更糟？

如果没有系统性的评估流程，我们只能依赖零散的“肉眼检查”或几道随便试试的“gut check”来判断，而这种主观方式不仅效率低，更无法发现隐藏的退化（regression）问题。

随着 LLM 应用规模和复杂度不断增加，评估（Evaluation）成为保障应用质量的关键环节。评估不是单纯为了“算一个分数”，它的根本价值在于：
- 提供一致、可量化的对比基准，确保每一版应用都在同一套标准上测试；
- 及时发现模型退化，避免提示词更新、模型替换或结构优化带来意外的质量下降；
- 帮助开发者理解改动的真实影响，为上线决策提供硬数据；
- 推动 LLM 应用进入工程化、可持续优化的循环。

因此，LangSmith 提供了一套完整的评估体系，让开发者可以真实、系统地回答以下问题：“随着应用迭代，我们真的在变得更好吗？”

这套体系由三个核心组件组成：
- Datasets — 用于构建稳定、可复用的测试集，让每次版本对比都公平透明。
- Evaluators — 根据输入、参考答案与模型输出计算各种评价指标，可以是用大模型来对特定组合进行评判，也可以设定特定规则来进行分析。
- Experiments — 将应用跑在数据集上，附加评估器，生成可视化的评分结果。

本质上这个实验打分的逻辑很简单，就是我们创建数据集（Dataset）里的每一条例子（Example）都是有输入（Input）和默认输出（Reference Output）。
那评估时，我们将会基于 Input 输入的问题，然后放到一个大模型的应用中运行（Run）并产生出结果（Output）。
然后评估器（Evaluator）将根据我们的设定的规则对比 Output 和 Reference Output 进行打分。

除了单轮的评估外，假如通过控制变量的方式进行了多轮的评估，我们可以在 LangSmith 上看到多轮结果的对比情况。
这样就能够帮助我们更直观地分析实验的结果，从而更快的找到最优的应用组合。

下面就通过一个实际的对话模型评估的例子来进行演示！

## 2.2 评估 ChatBot 应用

基于前面学习到的内容，假如我们要评估一个对话应用的话，我们开展的步骤为：
- 明确评估的目的（要改进什么，是准确率还是幻觉？怎么判断变好了还是变坏了？）
- 根据评估目的准备测试的数据集（Dataset）
- 根据评估目的搭建评估器（Evaluator）
- 准备对话应用所需的代码
- 启动实验（Experiment）
- 查看评估结果
- 更改提示词或模型再次启动实验（Experiment）
- 对比多次评估结果找到最优解

### 2.2.1 明确评估的目的

在搭建 LLM 应用的评估体系之前，我们都需要考虑清楚我们为什么需要做评估？

这个“为什么”，绝不仅仅是形式上的流程要求，而是整个评估体系最核心的起点。如果没有这个起点，评估就会变成盲目地跑数据、堆指标，看似“工作很充分”，但实际上并不能解决真实问题。

比如对于 ChatBot 类应用，我们评估的目的主要看以下几点：
- 评估对话质量：自然度、连贯性、逻辑性等
- 评估模型幻觉：输出内容是否是正确的，而不是一本正经的胡说八道
- 评估任务完成度：是否能够正确回复用户提出的问题
- 安全性与合规性：是否输出了不当内容（偏见、歧视、暴力等）

总的来说，ChatBot 的评估目的是确保它能稳定、准确、安全、且符合目标地完成任务。

### 2.2.2 准备测试的数据集（Dataset）

确定了评估的目的后，这里我们就可以准备一些数据集来测试模型了。对于真实项目而言，准备 10 - 50 个样例就已经足够了。我们可以通过手动收集或者让 AI 通过格式化输出生成对应的数据集，甚至还可以在项目上线后根据真实的用例进一步补充。

在我们这个例子中，我们就通过三条简单的常识性问题进行演示上传方式：

In [11]:
from langsmith import Client
client = Client() # 创建 LangSmith 客户端
dataset_name = "Chatbot Evaluation Dataset"
datasets = list(client.list_datasets(dataset_name=dataset_name))
if datasets:
    dataset = datasets[0]
    print(f"Dataset 已存在，直接使用: {dataset.id}")
else:
    dataset = client.create_dataset(dataset_name) # 创建一个数据集（如果已存在不会重复创建）
    # 向其中加入几条简单测试数据
    client.create_examples(
    dataset_id=dataset.id,
    examples=[{"inputs": {"question": "你是谁？"}, "outputs": {"answer": "我是一个智能助手。"}},
        {"inputs": {"question": "中国的首都是哪里？"}, "outputs": {"answer": "北京。"}},
        {"inputs": {"question": "1+1等于几？"}, "outputs": {"answer": "2"}}])

Dataset 已存在，直接使用: 8d3c5b0f-a5c5-4b70-8df2-b3a0c38bade9


### 2.2.3 搭建评估器（Evaluator）

评估器的搭建其实是基于评估目的来展开的。通常来说在 LangSmith 里常见的有四种评估器可以使用：
- LLM-as-Judge（大模型评分器）：让另一个大模型来判断表现是否好（最常见）
- Code Evaluators（代码评估器）：通过硬规则判断的“程序化评估器”
- Prebuilt Evaluators（预构建评估器）：LangSmith 内置的评估器
- Composite Evaluators（组合评估器）：多评估器组合使用

搭建评估器的方式其实很简单，本质上就是写一个函数，输入的内容包括问题、预设好的答案和实际模型输出的答案。输出就是对应的分数。目前最常见的方式就是使用另外一个大模型来作为判官来进行打分。

假如是用大模型来进行审阅的话，我们需要把前面三个输入传入用户提示词中，并且也告诉其判卷的规则即可。

然后假如模型返回的是“正确”，那么上传上去的结果就是 True，假如返回“不正确”的话，上传的结果就是 False 了。

In [12]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai
import os

judge_client = wrap_openai(OpenAI(
  api_key=os.getenv("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3"
))

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
  prompt = f"""
问题：{inputs['question']}
参考答案：{reference_outputs['answer']}
模型回答：{outputs['response']}

请判断模型回答是否正确，只回复“正确”或“不正确”。
"""
  resp = judge_client.chat.completions.create(
    model="ernie-3.5-8k",
    temperature=0,
    messages=[
      {"role": "system", "content": "你是一位专业中文评分老师。"},
      {"role": "user", "content": prompt}
    ]
  ).choices[0].message.content.strip()

  return resp == "正确"

除此之外，我们还可以去自定义一些个性化的评估器，比如了解模型的简洁度：

In [13]:
def concision(outputs: dict, reference_outputs: dict) -> bool:
  return int(len(outputs["response"]) <= 2 * len(reference_outputs["answer"]))

### 2.2.4 准备对话应用所需的代码

设置好了评估器后，我们也要准备一下可运行的 ChatBot 代码。这里为了演示我们使用的就是前面 wrap_openai 时大模型调用的代码进行演示：

In [14]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai

raw_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")
traced_client = wrap_openai(raw_client)

def my_chatbot(question: str) -> str:
  resp = traced_client.chat.completions.create(
    model="ernie-3.5-8k",
    messages=[{"role": "system", "content": "你是一个简洁中文助手"},
      {"role": "user", "content": question},])
  return resp.choices[0].message.content

### 2.2.5 启动实验（Experiment）

最后，就是把前面做好的数据集、评估器以及对话应用的代码进行组合，并开启实验。

这里需要注意的是，LangSmith 内部有要求的输入输出格式，所以我们要单独设置一个 ls_target 函数完成转换。

最后通过向 client.evaluate() 转换函数、数据集名称、评估器以及一个前缀 experiment_prefix（默认为 None）。这样就可以开启训练了。

In [15]:
from langsmith import Client
client = Client()

# LangSmith 需要 inputs→outputs 格式
def ls_target(inputs: dict) -> dict:
  return {"response": my_chatbot(inputs["question"])}

dataset_name = "Chatbot Evaluation Dataset"

# 运行 LangSmith 实验
experiment = client.evaluate(
  ls_target,
  data=dataset_name,
  evaluators=[correctness, concision],
  experiment_prefix="ernie-chatbot-eval"
)

print("评估完成")

View the evaluation results for experiment: 'ernie-chatbot-eval-62c8c6b0' at:
https://smith.langchain.com/o/85eb9f0e-8199-5590-8bca-f3805e58cb36/datasets/8d3c5b0f-a5c5-4b70-8df2-b3a0c38bade9/compare?selectedSessions=5456dff8-0150-4461-8da0-d5b870a083f7




3it [00:09,  3.13s/it]

评估完成


### 2.2.6 更改提示词或模型再次启动实验（Experiment）

对于单次的结果我们并不能有所对比找到最优解，我们可以尝试更新对话应用所需的代码，比如更换新的模型来重新进测试：

In [16]:
def my_chatbot(question: str) -> str:
  resp = traced_client.chat.completions.create(
    model="ernie-4.5-turbo-128k-preview",
    messages=[{"role": "system", "content": "你是一个简洁中文助手"},
      {"role": "user", "content": question},])
  return resp.choices[0].message.content

from langsmith import Client
client = Client()

# LangSmith 需要 inputs→outputs 格式
def ls_target(inputs: dict) -> dict:
  return {"response": my_chatbot(inputs["question"])}

dataset_name = "Chatbot Evaluation Dataset"

# 运行 LangSmith 实验
experiment = client.evaluate(
  ls_target,
  data=dataset_name,
  evaluators=[correctness, concision],
  experiment_prefix="ernie-chatbot-eval"
)

print("评估完成")

View the evaluation results for experiment: 'ernie-chatbot-eval-75f36314' at:
https://smith.langchain.com/o/85eb9f0e-8199-5590-8bca-f3805e58cb36/datasets/8d3c5b0f-a5c5-4b70-8df2-b3a0c38bade9/compare?selectedSessions=50f83d8a-1b5f-4720-82b6-6a1ce89d3ce9




3it [00:06,  2.07s/it]

评估完成


## 2.3 评估 RAG 应用

除了评估 ChatBot 以外，下面我们再来讲一下如何来评估 RAG 应用。

对于 RAG 应用而言，除了评估模型回复的内容是否合理以外，其实还有一个很重要的评估内容是向量数据库里检索到的片段是否真的相关。

下面我们就以一篇博客文章为例子看看如何对 RAG 应用进行评估吧！

和开展的步骤类似，但是对于 RAG 应用而言，我们还需要添加生成向量数据库的步骤：
- 明确评估的目的
- 生成向量数据库并准备对话应用所需的代码
- 根据评估目的准备测试的数据集（Dataset）
- 根据评估目的搭建评估器（Evaluator）
- 启动实验（Experiment）
- 查看评估结果

### 2.3.1 明确评估的目的

对于 RAG 应用而言，其主要的评估目的包括：
- 确认答案是否正确（Correctness）：对比一下提问的问题和回复的答案，看是否两者有所偏差。
- 检查回答是否切题、有帮助（Relevance）：对比一下模型回复的答案和数据集上的参考答案，看两者是否有很大的差异。
- 检查回答是否“立足于文档”（Groundedness）：当我们从向量数据库里检索到了相关内容，但模型可能内部的知识和文档有冲突，所以这里就是检查模型的回复是否按照向量数据库里检索到的最新知识进行回复。
- 检索的文档是否相关（Retrieval Relevance）：很多时候模型回复差的原因可能是检索不到有用的文档，所以提取出对应的文档内容以及提问的内容来进行对比分析。

### 2.3.2 生成向量数据库
这里我们使用的就是 lilianweng 发布在 github 上的三篇 blog 作为原始的数据（与 ReAct Agent 主题相关），并且将这部分内容切分后存放到向量数据库中（InMemoryVectorStore）：

In [17]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

urls = ["https://lilianweng.github.io/posts/2023-06-23-agent/",
  "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
  "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"]

docs = [WebBaseLoader(url).load() for url in urls]
docs = [d for batch in docs for d in batch]

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=250,
  chunk_overlap=0
)

splits = text_splitter.split_documents(docs)

from langchain_community.embeddings import DashScopeEmbeddings

vectorstore = InMemoryVectorStore.from_documents(
  splits,
  embedding=DashScopeEmbeddings(
    dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model="text-embedding-v1"
  )
)
retriever = vectorstore.as_retriever(k=6) # 找 6 篇相关的内容

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 2.3.3 准备对话应用所需的代码

然后我们可以顺手把对话的 RAG 应用也进行创建。运行文件后可以得到以下结果：

In [18]:
from langsmith import traceable
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="ernie-3.5-8k",
 openai_api_key=os.environ.get("OPENAI_API_KEY"),
 base_url="https://aistudio.baidu.com/llm/lmapi/v3")

@traceable()
def rag_bot(question: str):
  docs = retriever.invoke(question)
  context = "\n".join(d.page_content for d in docs)
  prompt = f"""
你是一名智能问答助手。请严格基于下列提供的参考文档内容回答用户问题：
====== 文档内容 ======
{context}
回答要求：
- 必须基于文档内容回答，不允许凭空补充
- 最多使用三句话
- 若文档中没有相关信息，请直接回答“文档未包含该信息”
"""
  answer = llm.invoke([{"role": "system", "content": prompt},
    {"role": "user", "content": question}])
  return {"answer": answer.content,
    "documents": docs}
if __name__ == "__main__": # 手动测试
  print(rag_bot("什么是 ReAct？"))

{'answer': 'ReAct (Yao et al. 2023) 是一种将推理和行动整合到大型语言模型（LLM）中的方法，通过扩展行动空间为任务特定离散行动和语言空间的组合。其提示模板包含明确的思考、行动和观察步骤，使模型能将反思与环境信息转化为行动。在知识密集型和决策任务实验中，ReAct表现优于仅行动的基线方法。', 'documents': [Document(id='ac4e5d82-e056-4997-b8a5-9e9dc6d80ec0', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient ha

### 2.3.4 准备测试的数据集

确定数据来源后，我们就可以进行数据集的创建，这里我们就通过三条数据进行演示：

In [19]:
from langsmith import Client
client = Client()

examples = [
  {"inputs": {"question": "ReAct 智能体是如何使用自我反思（self-reflection）机制的？"},
  "outputs": {"answer": "ReAct 将推理与行动结合，通过执行动作（如搜索信息）、观察反馈，再基于观察结果进行自我反思，从而优化后续推理步骤。"}},
  {"inputs": {"question": "在进行 few-shot 提示时可能产生哪些偏差？"},
  "outputs": {"answer": "常见偏差包括多数标签偏差、位置偏差（如最近示例影响更大）以及常见词偏差等。"}},
  {"inputs": {"question": "常见的五种对抗性攻击有哪些？"},
  "outputs": {"answer": "包括：输入词扰动、基于梯度的攻击、越狱提示（jailbreak）攻击、人类红队攻击以及模型红队攻击。"}}
]

dataset_name = "Lilian Weng 博客问答数据集"
if not client.has_dataset(dataset_name=dataset_name): # 如果不存在，就创建
  dataset = client.create_dataset(dataset_name=dataset_name)
  client.create_examples(dataset_id=dataset.id, examples=examples)
  print("Dataset created!")
else:
  print("Dataset already exists.")

Dataset already exists.


### 2.3.5 搭建评估器
根据评估目的，我们可以为该任务创建四个单独的评估器进行评估。

这里我们主要是通过大模型对比来进行评分，因此我们需要先设置一个大模型进行评分：

In [20]:
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="ernie-4.5-turbo-128k-preview",
  openai_api_key=os.environ.get("OPENAI_API_KEY"),
  base_url="https://aistudio.baidu.com/llm/lmapi/v3")

第一个我们搭建的评估器是正确性 Correctness，其主要是通过对比模型回复的内容和数据集中的输入和输出来进行评估，因此设置的方式如下：

In [21]:
from typing_extensions import Annotated, TypedDict
class CorrectnessGrade(TypedDict):
  explanation: Annotated[str, ..., "请解释你的判断依据"]
  correct: Annotated[bool, ..., "若学生回答与标准答案一致则为 True"]

correctness_llm = llm.with_structured_output(CorrectnessGrade, method="json_schema", strict=True) # 格式化输出结果

correctness_instructions = """你将收到：问题（QUESTION）、标准答案（GROUND TRUTH ANSWER）、学生答案（STUDENT ANSWER）。
你的任务：只评估事实正确性。并且最后输出布尔值 True 或 False。
要求：
- 如果学生答案与标准答案在事实内容上相符，即使有额外补充，也应视为正确。
- 如果学生答案出现事实性错误或与标准答案冲突，应评为错误。"""

def correctness(inputs, outputs, reference_outputs):
  text = f"""问题：{inputs['question']}
标准答案：{reference_outputs['answer']}
学生答案：{outputs['answer']}"""
  grade = correctness_llm.invoke([
    {"role": "system", "content": correctness_instructions},
    {"role": "user", "content": text}])
  return grade["correct"]

第二个评估器是关于相关性 Relevance，主要是对比输入模型的问题和回复的答案看两者是否相关。由于格式化输出的设置，所以返回的值必须是布尔值，也就是 True 或 False：

In [22]:
class RelevanceGrade(TypedDict):
  explanation: Annotated[str, ...]
  relevant: Annotated[bool, ...]

relevance_instructions = """
你将收到：问题（QUESTION）和学生答案（STUDENT ANSWER）。
请判断学生的回答是否与问题相关、是否有帮助。
"""

relevance_llm = llm.with_structured_output(
  RelevanceGrade, method="json_schema", strict=True
)

def relevance(inputs, outputs):
  text = f"问题：{inputs['question']}\n学生答案：{outputs['answer']}"
  grade = relevance_llm.invoke([
    {"role": "system", "content": relevance_instructions},
    {"role": "user", "content": text}
  ])
  return grade["relevant"]

第三个评估器是检查回答是否“立足于文档”（Groundedness），也就是通过搜索到的文档以及模型的回复来确定两者是否存在关联：

In [23]:
class GroundedGrade(TypedDict):
  explanation: Annotated[str, ...]
  grounded: Annotated[bool, ...]

grounded_instructions = """
你将收到：事实文档（FACTS）和学生答案（STUDENT ANSWER）。
你的任务：判断学生的回答是否完全基于文档提供的事实，而非凭空杜撰。
"""

grounded_llm = llm.with_structured_output(
  GroundedGrade, method="json_schema", strict=True
)

def groundedness(inputs, outputs):
  docs = "\n\n".join(doc.page_content for doc in outputs["documents"])
  text = f"事实文档（FACTS）：\n{docs}\n\n学生答案（STUDENT ANSWER）：\n{outputs['answer']}"
  grade = grounded_llm.invoke([
    {"role": "system", "content": grounded_instructions},
    {"role": "user", "content": text}
  ])
  return grade["grounded"]

最后一个评估器是检查检索到的文档是否与问题相关（retrieval_relevance）：

In [24]:
class RetrievalRelGrade(TypedDict):
  explanation: Annotated[str, ...]
  relevant: Annotated[bool, ...]

retrieval_instructions = """
你将收到：问题（QUESTION）与检索到的文档（RETRIEVED FACTS）。
任务：判断这些文档是否包含回答问题所需的关键信息。
"""

retrieval_llm = llm.with_structured_output(
  RetrievalRelGrade, method="json_schema", strict=True
)

def retrieval_relevance(inputs, outputs):
  docs = "\n\n".join(doc.page_content for doc in outputs["documents"])
  text = f"问题（QUESTION）：\n{inputs['question']}\n\n检索文档（FACTS）：\n{docs}"
  grade = retrieval_llm.invoke([
    {"role": "system", "content": retrieval_instructions},
    {"role": "user", "content": text}
  ])
  return grade["relevant"]

### 2.3.6 启动试验
在所有配件准备好后，我们就可以基于前面的内容开始进行测试了：


In [25]:
from langsmith import Client

client = Client()


# 与 create_rag_dataset.py 中保持一致
dataset_name = "Lilian Weng 博客问答数据集"

def target(inputs: dict) -> dict:
  return rag_bot(inputs["question"])

experiment = client.evaluate(
  target,      # 你的 RAG 系统（评估对象）
  data=dataset_name, # 数据集名称（字符串即可）
  evaluators=[    # 四类 RAG 自动评估维度
    correctness,    # 正确性：回答是否与标准答案一致
    groundedness,    # 依据性：回答是否基于检索文档
    relevance,     # 相关性：回答对用户是否有帮助
    retrieval_relevance # 检索相关性：检索到的文档是否与问题匹配
  ],
  experiment_prefix="rag-文档匹配评估", # UI 中的实验名称前缀
  metadata={
    "version": "ernie-rag-v1", # 可用于标注你的系统版本
    "description": "ERNIE 模型 + 本地向量库的 RAG 系统评估实验"
  },
)

print("✅ RAG 自动评估已完成！")

View the evaluation results for experiment: 'rag-文档匹配评估-34f004d6' at:
https://smith.langchain.com/o/85eb9f0e-8199-5590-8bca-f3805e58cb36/datasets/cde3efa7-5844-4aab-995b-669c6d5277b7/compare?selectedSessions=5b0dec33-ce9f-411a-b337-85579c0bc45f




3it [00:39, 13.16s/it]

✅ RAG 自动评估已完成！


### 2.3.7 查看评估结果

在页面中就能看到完整的测试结果了。结果显示除了 Groundedness 以外，其他的结果都并不那么相关，所以我们可以对其进行优化并获取结果。